<a href="https://colab.research.google.com/github/hariexcel/ACE_Framework/blob/main/Make_it_More_PUBLIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Make it More, by Matt Shumer (https://twitter.com/mattshumer_)
prompt = 'very spicy ramen' #@param {type:"string"}
openai_key = '' #@param {type:"string"}
steps = 5 #@param {type:"slider", min:1, max:10, step:1}

!pip install --upgrade openai > /dev/null

from openai import OpenAI
from PIL import Image
import requests
from io import BytesIO
import requests

client = OpenAI(api_key = openai_key)

def decide_natural_or_vivid(prompt):
    # Pass the image to GPT-4 Vision API to get suggestions
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "Given a prompt to generate an image, decide which type of image it should be: `natural` or `art`. `art` is more artistic, `natural` is more realistic."
            },
            {
                "role": "user",
                "content": "Prompt: `{prompt}`"
            },
        ],
        logit_bias = {
            '53980': 100, # 'natural' token
            '472': 100, # 'art' token
        },
        max_tokens=1,
        temperature=0,
    )

    # Get the suggestion
    output = response.choices[0].message.content
    print('Style Chosen:', output)

    if 'art' in output:
      return 'vivid'
    elif 'natural' in output:
      return 'natural'

def generate_image(prompt, style):
    # Generate an image with DALL-E
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="hd",
        style=style,
        n=1,
    )

    # Get the generated image URL
    image_url = response.data[0].url

    revised_prompt = response.data[0].revised_prompt

    return image_url, revised_prompt


def get_artistic_suggestion(image_url, prompt):
    # Pass the image to GPT-4 Vision API to get suggestions
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "system",
                "content": """Your goal is to, given an image and the prompt used to create it, give suggestions to 'make it more'.

Here is an overview of the "Make it More" trend:
The "Make It More" trend is a creative phenomenon that has emerged within the AI community, where users push the boundaries of AI image generation capabilities. The process begins with an initial image, often simple or innocuous, which is then fed into an AI system like ChatGPT. Users then instruct the AI to amplify or exaggerate the image, often with the phrase "make it more".

The AI then iteratively enhances and intensifies the image, often resulting in absurd, surreal, or wildly imaginative outputs that far exceed the original concept. For instance, a simple image of a dog could be transformed into an ecstatic cosmic canine, or a bowl of ramen could end up as a fiery dish setting the universe ablaze.

This trend not only showcases the creative potential of AI but also offers a glimpse into how AI works to satisfy our whims. It's more than just a meme trend; it's a unique, self-sustaining loop where the peculiarities and capabilities of AI art generation tools are both the medium and the message. It's a new form of digital culture, with its own in-jokes, norms, and memes, all centered around AI technology.

Examples:
1. A simple flower: Start with a basic image of a flower. Ask the AI to make it more. It might turn the flower into a glowing, radiant blossom emitting rays of light. Ask for more, and the AI might transform the flower into a colossal, cosmic entity, with petals made of galaxies and a core of pure starlight.

2. A cup of coffee: Begin with a regular cup of coffee. Ask the AI to make it more. It could make the coffee steaming hot, with the aroma visibly wafting off the cup. Request for more, and the AI could create a cup of coffee that's literally brewing a storm, with lightning bolts crackling in the steam.

3. A city skyline: Start with a simple city skyline. Ask the AI to make it more. It might add more skyscrapers, lights, and even futuristic structures. Request for more, and the AI could transform the city into a sprawling metropolis that extends into the sky, with buildings that touch the stars and roads made of clouds.

4. A cat: Begin with a cute image of a cat. Ask the AI to make it more. It could turn the cat into a majestic, royal feline with a crown and a cape. Ask for more, and the AI might transform the cat into a mythical creature, with wings of a dragon, the aura of a phoenix, and the majesty of a tiger.

5. A pizza: Start with a simple pizza. Ask the AI to make it more. It might add more toppings, cheese, and even make it a double-decker. Request for more, and the AI could create a pizza that's a full-on feast, with layers of different cuisines, a crust made of golden bread, and cheese that's melting like a waterfall."""
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Here was the previous prompt that generated the image you see: `{prompt.strip()}`. In two sentences or less, how can we 'make it more'?"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        },
                    },
                ],
            }
        ],
        max_tokens=250,
    )

    # Get the suggestion
    suggestion = response.choices[0].message.content
    print('Suggestion:', suggestion)

    # Create a new prompt
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "system",
                "content": "You are a system designed to generate prompts for image generation models. Always do exactly as you are told."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Here is the current prompt that was used to generate this image: `{prompt}`.\n\nHere is a suggestion for improvement: `{suggestion}`.\n\nWrite a new prompt that incorporates the suggestion to generate a better image. Make it clear, and three sentences or less. Don't mention that there have been other prompts, just describe what you want the new image to look like."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        },
                    },
                ],
            }
        ],
        max_tokens=250,
    )

    # Get the improved prompt
    improved_prompt = response.choices[0].message.content

    print('Improved Prompt:', improved_prompt)

    return improved_prompt


def gpt_artist(prompt, steps):
    style = decide_natural_or_vivid(prompt)

    for i in range(steps):
        try:
          # Generate an image
          image_url, prompt = generate_image(prompt, style)

          # Show the image
          response = requests.get(image_url)
          img = Image.open(BytesIO(response.content))
          display(img)

          # Get artistic suggestion
          prompt = get_artistic_suggestion(image_url, prompt)
        except:
          try:
            # Generate an image
            image_url, prompt = generate_image(prompt, style)

            # Show the image
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            display(img)

            # Get artistic suggestion
            prompt = get_artistic_suggestion(image_url, prompt)
          except:
            # Generate an image
            image_url, prompt = generate_image(prompt, style)

            # Show the image
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            display(img)

            # Get artistic suggestion
            prompt = get_artistic_suggestion(image_url, prompt)

    return prompt

# Test the 'gpt-artist'
final_prompt = gpt_artist(prompt, steps)
print("Final prompt:", final_prompt)